# FB2NEP Workbook 5 – Data Transformation and Preparation for Modelling

We consider:

- Why we transform data (skewness, comparability, interpretability).
- Log transformation and z-scoring.
- Pitfalls of categorisation.

Run the first two code cells to set up the repository and load the data.

In [ ]:
# FB2NEP bootstrap cell – use in *all* workbooks

import pathlib
import runpy

bootstrap_candidates = [
    "scripts/bootstrap.py",
    "../scripts/bootstrap.py",
    "../../scripts/bootstrap.py",
]

bootstrap_ns = None

for rel in bootstrap_candidates:
    p = pathlib.Path(rel)
    if p.exists():
        print(f"Loading bootstrap from: {p}")
        bootstrap_ns = runpy.run_path(str(p))
        break
else:
    raise FileNotFoundError(
        "Could not find scripts/bootstrap.py. "
        "Please check that you are running this notebook inside fb2nep-epi."
    )

if "init" not in bootstrap_ns:
    raise RuntimeError("bootstrap.py does not define init().")

df, CTX = bootstrap_ns["init"]()

REPO_ROOT = CTX.repo_root
CSV_REL = CTX.csv_rel
IN_COLAB = CTX.in_colab

print("Repository root:", REPO_ROOT)
print("Main dataset:", CSV_REL)
print("df shape:", df.shape)
print("IN_COLAB:", IN_COLAB)

In [ ]:
import pandas as pd

# Load the main synthetic cohort used in all FB2NEP workbooks
df = pd.read_csv("data/synthetic/fb2nep.csv")

# Quick check: first rows
df.head()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
%matplotlib inline

## 1. Skewness and transformation

We inspect the distribution of a skewed variable, for example `red_meat_g_d`.

In [ ]:
var = "red_meat_g_d" if "red_meat_g_d" in df.columns else "energy_kcal"
plt.figure(figsize=(6, 4))
df[var].hist(bins=30)
plt.xlabel(var)
plt.ylabel("Number of participants")
plt.title(f"Distribution of {var}")
plt.tight_layout()
plt.show()
print("Skewness:", stats.skew(df[var].dropna()))

## 2. Log transformation

We apply a simple log transformation to reduce skewness.

In [ ]:
def log_transform(x: pd.Series, constant: float = 0.0) -> pd.Series:
    return np.log(x + constant)

df["log_" + var] = log_transform(df[var], constant=0.1)
plt.figure(figsize=(6, 4))
df["log_" + var].hist(bins=30)
plt.xlabel(f"log({var})")
plt.ylabel("Number of participants")
plt.title(f"Distribution of log-transformed {var}")
plt.tight_layout()
plt.show()

## 3. Standardisation (z-scores)

We standardise BMI and SBP to have mean 0 and standard deviation 1.

In [ ]:
def z_score(x: pd.Series) -> pd.Series:
    return (x - x.mean()) / x.std()

for v in ["BMI", "SBP"]:
    if v in df.columns:
        df["z_" + v] = z_score(df[v])
        print(f"\nSummary of z-scored {v}:")
        display(df["z_" + v].describe())

## 4. Categorisation and information loss

We compare continuous BMI with BMI tertiles.

In [ ]:
if {"BMI", "SBP"}.issubset(df.columns):
    df["BMI_tertile"] = pd.qcut(df["BMI"], 3, labels=["low", "medium", "high"])
    plt.figure(figsize=(6, 4))
    plt.scatter(df["BMI"], df["SBP"], alpha=0.3)
    plt.xlabel("BMI (kg/m²)")
    plt.ylabel("SBP (mmHg)")
    plt.title("SBP vs continuous BMI")
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(6, 4))
    df.boxplot(column="SBP", by="BMI_tertile")
    plt.xlabel("BMI tertile")
    plt.ylabel("SBP (mmHg)")
    plt.title("SBP by BMI tertile")
    plt.suptitle("")
    plt.tight_layout()
    plt.show()